In [ ]:
!pip install qrcode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 4.2 MB/s eta 0:00:00


In [ ]:
'''
digital-rock-expert
Takes a CSV of hit songs, and allows you to play a game where a song is
randomly selected and played, and the players must guess the year, artist, and
title.

Author - Steve Holl
Initial Release August 2024
'''

In [ ]:
import sys
import os
import requests
import csv
import io
import re
from datetime import datetime
from time import sleep
from tabulate import tabulate

import qrcode
import pandas as pd

from IPython.display import display
from IPython.display import Audio

In [ ]:
FILENAME = "/content/rock-chart-final.csv"

In [ ]:
def read_hits_csv(filename=FILENAME):
  '''Loads a hits csv into a dataframe for playing. Expects column names:
  year, rank, artist, title, and spotify_url.'''

  df = pd.read_csv(filename)
  df = df[df['spotify_url'].notnull()]
  while filename == "":
    input('Please enter the full path of the CSV where the hits are located.')

  df = pd.read_csv(filename)
  print(f'{len(df)} hits loaded from {filename}!')

  return df

In [ ]:
def get_year_range(df):
  '''Trims the hit list to specified years and top N songs'''

  try:
    earliest = input(f'What is the earliest year you want to use? File contains {df.year.min()} to {df.year.max()} (e.g. {df.year.min()}): ')
  except:
    earliest = df.year.min()
    print('Invalid input. Setting year to the minimum year available in the uploaded data.')

  try:
    latest = input(f'What is the latest year you want to use? File contains {df.year.min()} to {df.year.max()} (e.g. {df.year.max()}): ')
  except:
    latest = df.year.max()
    print('Invalid input. Setting year to the maximum year available in the uploaded data.')

  df = df[(df['year'] >= int(earliest)) & (df['year'] <= int(latest))]

  print(f'You have selected a range of {int(latest) - int(earliest)} years. This contains {len(df)} songs.')
  top = 50
  top = input('Would you like to reduce to a certain amount of top hits per year? If so, enter the amount of top hits per year to play with (e.g. 10): ')
  df = df[(df['rank'] >= int(1)) & (df['rank'] <= int(top))]

  return df

In [ ]:
def team_init():
  '''Gets team names and sets up the team list'''
  team_cnt = input('\n\nHow many teams are playing? (e.g. 2): ')
  print('\n')

  teams = []

  for team in range(1,int(team_cnt)+1):
    name = ""
    while name == "":
      name = input(f'Enter the team {team} name: ')
    teams.append(name)
  return teams

In [ ]:
def generate_qr_code(url):
    '''Generates a QR code for the URL.'''

    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_L,
        box_size=10,
        border=4,
    )

    qr.add_data(url)
    qr.make(fit=True)

    img = qr.make_image(fill='black', back_color='white')

    return img

In [ ]:
def get_song(df,boards,team_idx):
  '''Selects a random song from the specified DataFrame'''

  my_song = df.sample(n=1)
  idx = my_song.index

  #Play song
  print('\nNow playing your song to guess.\n\n')

  my_qr = generate_qr_code(my_song['spotify_url'].values[0])

  display(my_qr)
  print(my_song['spotify_url'].values[0])

  input('\nPress enter to find out song details.')
  print(tabulate(my_song[['year','artist','title']],
                headers='keys', tablefmt='psql',showindex=False))

  df.drop(idx, inplace=True)

  global teams
  curr_team = teams[team_idx]

  result = input(f'Did team {curr_team} guess correctly? (e.g. y, n) ')

  if result.lower() == 'y' or result.lower() == 'yes':
    boards[team_idx] = pd.concat([boards[team_idx], my_song], ignore_index=True)
  else:
    print ('\n')
    is_stolen = input('Did another player steal the guess? (y/n): ')
    if is_stolen.lower() == 'y' or result.lower() == 'yes':
      for i in range(0,len(teams)):
        print(f'{i}. {teams[i]}')

      stealing_idx = ""
      while stealing_idx == "":
        x = input('Enter the number of the team that won the steal: ')
        try:
          stealing_idx = int(x)
        except:
          stealing_idx = ""

      boards[stealing_idx] = pd.concat([boards[stealing_idx], my_song],
                                        ignore_index=True)

  return df,boards

In [ ]:
def board_init(teams,df):
  '''When passed the team board list  and song df, sets up the board for
   each team with a randomly selected song.'''

  boards = []

  for team in range(0,len(teams)):
    boards.append(None)

  for team in range(0,len(teams)):
    my_song = df.sample(n=1)
    idx = my_song.index
    df.drop(idx, inplace=True)
    boards[team] = pd.DataFrame(my_song)
  return boards

In [ ]:
#Initialize game
df = read_hits_csv()
teams = team_init()
df = get_year_range(df)
boards = board_init(teams,df)

def main():

  print('======================================\n')

  # Start game

  idx = 0

  a = pd.DataFrame()
  winning_team = ""

  global df
  global teams
  global boards

  while len(df) > 0 and winning_team == "":

      print('Current team scores:\n====================')
      for i in range(0,len(teams)):
        print(f'{teams[i]}: {len(boards[i])}')
        if len(boards[i]) >= 10:
          winning_team = teams[i]
      print('====================\n\n')
      if winning_team != "":
          print(f'\n\n****************************************\n \
          {winning_team} has won!\n****************************************\n\n')
          break

      curr_team = teams[idx]
      print(f'Team {curr_team} is now playing.\n')
      print('This is your current board:')
      print(tabulate(boards[idx][['year','artist','title']],
                    headers='keys', tablefmt='psql',showindex=False))

      df,boards = get_song(df,boards,idx)

      # Move to the next team, wrapping around at end
      idx = (idx + 1) % len(teams)

      print('\n====================================\n')

if __name__ == "__main__":
    main()


6045 hits loaded from /content/rock-chart-final.csv!


